In [ ]:
# --- IMPORT ---
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from mpl_toolkits.mplot3d import Axes3D

sns.set(style='whitegrid')

# --- PERCORSI CSV ---
file_paths = {
    'a': Path('case_a.csv'),
    'b': Path('case_b.csv'),
    'c': Path('case_c.csv'),
    'd': Path('case_d.csv')
}

dfs = {}
T1 = {}

# --- CARICAMENTO E PREPROCESSING ---
for key, path in file_paths.items():
    df = pd.read_csv(path)
    if key == 'a':
        T1[key] = df.iloc[0]['time']
        df = df.iloc[1:].copy().reset_index(drop=True)
    else:
        T1[key] = None
        df = df.copy().reset_index(drop=True)

    df['n_thread_tot'] = df['n_thread'] * df['n_block']
    df['GFlops'] = 2 * df['M'] * df['N'] * df['K'] / (df['time'] * 1e9)

    if key == 'a':
        df['speedup'] = T1[key] / df['time']
    else:
        grouped = df.groupby('method')
        df['speedup'] = grouped['time'].transform(lambda x: x.iloc[0] / x)
    
    df['efficiency'] = df['speedup'] / (df['n_proc'] * df['n_thread'])
    dfs[key] = df

# --- FUNZIONE PER PLOTTARE ---
def plot_by_method(df, x, y, title, xlabel, ylabel):
    plt.figure(figsize=(8, 5))
    sns.lineplot(data=df, x=x, y=y, hue='method', style='method', marker='o')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()

# --- a.1 ---
print("Caso a.1")
df_a = dfs['a']
ref_n_thread = df_a['n_thread_tot'].mode()[0]
sub_a1 = df_a[df_a['n_thread_tot'] == ref_n_thread]

plot_by_method(sub_a1, 'n_proc', 'time', 'Caso a.1 - Tempo vs n_proc', 'n_proc', 'Tempo (s)')
plot_by_method(sub_a1, 'n_proc', 'speedup', 'Caso a.1 - Speed-up vs n_proc', 'n_proc', 'Speed-up')
plot_by_method(sub_a1, 'n_proc', 'efficiency', 'Caso a.1 - Efficienza vs n_proc', 'n_proc', 'Efficienza')

# --- a.2 ---
print("Caso a.2")
ref_n_proc = df_a['n_proc'].mode()[0]
sub_a2 = df_a[df_a['n_proc'] == ref_n_proc]

plot_by_method(sub_a2, 'n_thread_tot', 'time', 'Caso a.2 - Tempo vs n_thread_tot', 'Thread totali', 'Tempo (s)')
plot_by_method(sub_a2, 'n_thread_tot', 'speedup', 'Caso a.2 - Speed-up vs n_thread_tot', 'Thread totali', 'Speed-up')
plot_by_method(sub_a2, 'n_thread_tot', 'efficiency', 'Caso a.2 - Efficienza vs n_thread_tot', 'Thread totali', 'Efficienza')

# --- a.3 ---
print("Caso a.3")
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_a['n_proc'], df_a['n_thread_tot'], df_a['time'], c='blue', marker='o')
ax.set_xlabel('n_proc')
ax.set_ylabel('n_thread_tot')
ax.set_zlabel('Tempo (s)')
plt.title('Caso a.3 - Tempo 3D')

best = df_a.loc[df_a['time'].idxmin()]
ax.scatter(best['n_proc'], best['n_thread_tot'], best['time'], color='red', s=100, label='Migliore')
ax.legend()
plt.show()

print("Configurazione ottimale caso a.3:")
print(best[['n_proc', 'n_block', 'n_thread', 'method', 'time']])

# --- CASI B, C, D ---
def analyze_case(df, case, xvar, xlab, fixed_filter=None):
    print(f"Caso {case.upper()}")
    if fixed_filter:
        for k, v in fixed_filter.items():
            df = df[df[k] == v]
    plot_by_method(df, xvar, 'time', f'Caso {case.upper()} - Tempo vs {xvar}', xlab, 'Tempo (s)')
    plot_by_method(df, xvar, 'speedup', f'Caso {case.upper()} - Speed-up vs {xvar}', xlab, 'Speed-up')
    plot_by_method(df, xvar, 'efficiency', f'Caso {case.upper()} - Efficienza vs {xvar}', xlab, 'Efficienza')

# --- b: tempo vs n_proc (dimensione per processo fissa) ---
analyze_case(dfs['b'], 'b', 'n_proc', 'n_proc')

# --- c: tempo vs n_thread_tot (n_proc fisso) ---
ref_proc_c = dfs['c']['n_proc'].mode()[0]
analyze_case(dfs['c'], 'c', 'n_thread_tot', 'Thread totali', {'n_proc': ref_proc_c})

# --- d: tempo vs n_proc (n_thread_tot fisso) ---
ref_threads_d = dfs['d']['n_thread_tot'].mode()[0]
analyze_case(dfs['d'], 'd', 'n_proc', 'n_proc', {'n_thread_tot': ref_threads_d})

# --- RIEPILOGO CONFIGURAZIONI MIGLIORI ---
print("\nConfigurazione migliore per ciascun caso:")
for key in ['a', 'b', 'c', 'd']:
    best = dfs[key].loc[dfs[key]['time'].idxmin()]
    print(f"\nCaso {key.upper()}:")
    print(best[['n_proc', 'n_block', 'n_thread', 'method', 'time', 'speedup', 'efficiency', 'GFlops']])